In [31]:
import numpy as np
                       
import matplotlib.pyplot as plt
%matplotlib inline

import os
import re

import pennylane as qml
from pennylane import expval, var

import torch
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F

In [2]:
%load_ext autoreload
%autoreload 2

## Constants

In [3]:
n_qubits = 2
n = n_qubits

In [4]:
data_folder = "data/{}q".format(n_qubits)

## Load dataset

In [5]:
circuit_file = os.path.join(data_folder, "circuits.txt")
energy_file = os.path.join(data_folder, "energies.txt")

In [6]:
energies = np.loadtxt(energy_file)

In [7]:
def parse_command(command):
    command = re.split("\(|\)|,", command)[:-1]
    for i in range(1, len(command)):
        command[i] = int(command[i])
    return command

In [8]:
dataset_commands = []
with open(circuit_file, "r") as f:
    list_lines = f.read().splitlines()
    for line in list_lines:
        dataset_commands.append(list(map(parse_command, line.split())))

## State preparation

In [182]:
dev = qml.device('default.qubit', wires=3)

### Definition of the circuit

In [341]:
def universal_block(params, q):
    qml.RZ(params[0], wires=q[0])
    qml.RY(params[1], wires=q[0])
    qml.RZ(params[2], wires=q[0])
    qml.RZ(params[3], wires=q[1])
    qml.RY(params[4], wires=q[1])
    qml.RZ(params[5], wires=q[1])
    qml.CNOT(wires=[q[1],q[0]])
    qml.RZ(params[6], wires=q[0])
    qml.RY(params[7], wires=q[1])
    qml.CNOT(wires=[q[0],q[1]])
    qml.RY(params[8], wires=q[1])
    qml.CNOT(wires=[q[1],q[0]])
    qml.RZ(params[9], wires=q[0])
    qml.RY(params[10], wires=q[0])
    qml.RZ(params[11], wires=q[0])
    qml.RZ(params[12], wires=q[1])
    qml.RY(params[13], wires=q[1])
    qml.RZ(params[14], wires=q[1])

In [342]:
@qml.qnode(dev, interface='torch')
def sp_network(sp_params):
    state_preparation(sp_params)
    return (qml.expval.PauliZ(1), qml.expval.PauliZ(2))

### Creation of the dataset

In [ ]:
dataset_sp_params = [np.rand]

In [318]:
# def state_preparation(list_commands):
#     print(list_commands)
#     for command in list_commands:
#         print(command[0].val)
#         if command[0].val == "cnot":
#             print(command[0].val, command[1].val, command[2].val)
#             qml.CNOT(wires=[command[1].val, command[2].val])
#         elif command[0].val == "hadamard":
#             print(command[0].val, command[1].val)
#             qml.Hadamard(wires=command[1].val)
#         elif command[0].val == "phase":
#             print(command[0].val, command[1].val)
#             qml.PhaseShift(np.pi/2, wires=command[1].val)
#         else:
#             raise "Non recognized command {}".format(command[0].val)

## Variational circuit

### Main blocks

In [311]:
def CCNOT(wires):
    U = np.eye(8)
    U[-2:, -2:] = np.array([[0, 1], [1, 0]])
    qml.QubitUnitary(U, wires=wires)

In [312]:
def controlled_block(params):
    qml.CRZ(params[0], wires=[0, 1])
    qml.CRY(params[1], wires=[0, 1])
    qml.CRZ(params[2], wires=[0, 1])
    
    qml.CRZ(params[3], wires=[0, 2])
    qml.CRY(params[4], wires=[0, 2])
    qml.CRZ(params[5], wires=[0, 2])
    
    CCNOT(wires=[0, 2, 1])
    qml.CRZ(params[6], wires=[0, 1])
    qml.CRY(params[7], wires=[0, 2])
    
    CCNOT(wires=[0, 1, 2])
    qml.CRY(params[8], wires=[0, 2])
    CCNOT(wires=[0, 2, 1])
    
    qml.CRZ(params[9], wires=[0, 1])
    qml.CRY(params[10], wires=[0, 1])
    qml.CRZ(params[11], wires=[0, 1])
    
    qml.CRZ(params[12], wires=[0, 2])
    qml.CRY(params[13], wires=[0, 2])
    qml.CRZ(params[14], wires=[0, 2])

In [337]:
@qml.qnode(dev, interface='torch')
def sp_network(sp_commands):
    state_preparation(sp_commands)
    return (qml.expval.PauliZ(1), qml.expval.PauliZ(2))

In [340]:
sp_network(dataset_commands[2])

tensor([-3.7494e-33,  0.0000e+00], dtype=torch.float64)

In [ ]:
def variational_circuit(params):
    qml.Hadamard(0)
    controlled_block(params)
    qml.Hadamard(0)

In [199]:
class EndNetwork(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EndNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 1) 
        
        for param in self.parameters():
            if len(param.shape) >= 2:
                nn.init.orthogonal_(param.data)
    
    def forward(self, x):
        out = x.float()
        out = self.fc1(out)

        return out

In [200]:
end_network = EndNetwork(1, 1)
end_params = end_network.parameters()

### Composition

In [204]:
@qml.qnode(dev, interface='torch')
def total_qnn(sp_commands, params):
    sp_network(sp_commands)
    variational_circuit(params)

    return qml.expval.PauliZ(0)

In [205]:
total_qnn(dataset_commands[0], qnn_params)

TypeError: CRZ: Real scalar parameter expected, got <class 'str'>.

In [142]:
def total_network(sp_commands, params):
    return end_network.forward(total_qnn(sp_commands, params))

In [195]:
qnn_params = torch.tensor(np.random.normal(size=15, scale=0.1), requires_grad=True)

In [144]:
total_network(dataset_commands[0], qnn_params)

[[<pennylane.variable.Variable object at 0x12481aad0>, <pennylane.variable.Variable object at 0x12481a210>], [<pennylane.variable.Variable object at 0x12481aa50>, <pennylane.variable.Variable object at 0x124374ed0>], [<pennylane.variable.Variable object at 0x124374b90>, <pennylane.variable.Variable object at 0x124374690>], [<pennylane.variable.Variable object at 0x12487f050>, <pennylane.variable.Variable object at 0x12487f090>], [<pennylane.variable.Variable object at 0x12487f0d0>, <pennylane.variable.Variable object at 0x1243741d0>], [<pennylane.variable.Variable object at 0x12487f110>, <pennylane.variable.Variable object at 0x12487f150>]]
-0.038364731560320414


TypeError: CRZ: Real scalar parameter expected, got <class 'str'>.

In [ ]:
dataset_commands[]